In [1]:
import getpass
import os

import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [2]:
from typing import Annotated
from typing_extensions import TypedDict


In [3]:
##Working wih external tools
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper,AskNewsAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun, AskNewsSearch 

In [4]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki_tool=WikipediaQueryRun(api_wrapper=wiki_wrapper)



In [5]:
wiki_tool.invoke("Who is sharukh khan")

'Page: Shah Rukh Khan\nSummary: Shah Rukh Khan (pronounced [ˈʃaːɦɾʊx xäːn] ; born 2 November 1965), also known by the initialism SRK, is an Indian actor and film producer who works in Hindi cinema. Referred to in the media as the "Baadshah of Bollywood" and "King Khan", he has appeared in more than 10'

In [6]:
tools=[wiki_tool,arxiv_tool]


In [7]:
from langgraph.graph.message import add_messages

#Langraph application
class State(TypedDict):
    messages:Annotated[list,add_messages]
    
    

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

c:\Users\vansh\OneDrive\Desktop\ai_pipeline_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
llm_with_wikipedia=llm.bind_tools(tools=tools)

In [10]:
def chatbot(state: State):
    return {"messages":[llm_with_wikipedia.invoke(state["messages"])]}

In [11]:
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode,tools_condition
grap_builder = StateGraph(State)
grap_builder.add_node("chatbot",chatbot)
grap_builder.add_edge(START,"chatbot")
tool_node=ToolNode(tools=tools)

grap_builder.add_node("tools",tool_node)

grap_builder.add_conditional_edges(
    "chatbot",
    tools_condition)

grap_builder.add_edge("tools","chatbot")
grap_builder.add_edge("chatbot",END)

In [12]:
graph=grap_builder.compile()

In [13]:
user_input="what is Transformeer reseach paper"

events=graph.stream(
    {"messages":[("user",user_input)]},stream_mode="values"
) 

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what is Transformeer reseach paper
================================== Ai Message ==================================
Tool Calls:
  arxiv (3d3aa4e8-57cd-40c9-bad3-bc0127a97328)
 Call ID: 3d3aa4e8-57cd-40c9-bad3-bc0127a97328
  Args:
    query: Transformer research paper
================================= Tool Message =================================
Name: arxiv

Published: 2024-05-27
Title: SoK: Leveraging Transformers for Malware Analysis
Authors: Pradip Kunwar, Kshitiz Aryal, Maanak Gupta, Mahmoud Abdelsalam, Elisa Bertino
Summary: The introduction of transformers has been an important breakthrough for AI
research and application as transformers are the f
================================== Ai Message ==================================

One relevant Transformer research paper is titled  "SoK: Leveraging Transformers for Malware Analysis" published on 2024-05-27 by Pradip Kunwar, Kshitiz Aryal, Maanak Gupta,